# Movie Data Analysis - Learn Python for Data Science Portfolio Project

In this project I will be analysing a dataset with information on several movies. The project will be organised in the following fashion:

1. **Data Analysis**<br>
    Importing, cleaning and inspecting the dataset
2. **Descriptive Statistics**<br>
    Various statistics about the columns and their values
3. **Value Summaries**<br>
    Summaries and correlation beween some values
4. **Profitability Analysis**<br>
    Inspecting movie profitability with more detail
5. **Star power**<br>
    Inspecting stars, writers, directors and correlations

### 1. Data analysis

Importing the libraries and then the data from the file to a dataframe:

In [2]:
import pandas as pd
import numpy as np

movie_data = pd.read_csv('movies_updated.csv')

Previewing the 5 first rows to make sure our data looks good

In [3]:
movie_data.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,"runtime,,"
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772.0,Warner Bros.,"146.0,"
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106.0,Columbia Pictures,"104.0,"
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067.0,Lucasfilm,"124.0,"
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539.0,Paramount Pictures,"88.0,"
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344.0,Orion Pictures,"98.0,"


Cheking number of rows, columns and looking for missing values:

In [20]:
print(f'Number of rows: {movie_data.shape[0]}\nNumber of columns: {movie_data.shape[1]}') 
movie_data.isna().sum()

Number of rows: 4000
Number of columns: 15


name           0
rating        40
genre          0
year           0
released       0
score          0
votes          0
director       0
writer         1
star           1
country        0
budget         0
gross        169
company       10
runtime,,      0
dtype: int64